<h2>K-fold predictions</h2>

In [1]:
from fastai.vision import *
from fastai.callbacks import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold

In [2]:
def _one_sample_positive_class_precisions(scores, truth):
    """Calculate precisions for each true class for a single sample.

    Args:
      scores: np.array of (num_classes,) giving the individual classifier scores.
      truth: np.array of (num_classes,) bools indicating which classes are true.

    Returns:
      pos_class_indices: np.array of indices of the true classes for this sample.
      pos_class_precisions: np.array of precisions corresponding to each of those
        classes.
    """
    num_classes = scores.shape[0]
    pos_class_indices = np.flatnonzero(truth > 0)
    # Only calculate precisions if there are some true classes.
    if not len(pos_class_indices):
        return pos_class_indices, np.zeros(0)
    # Retrieval list of classes for this sample.
    retrieved_classes = np.argsort(scores)[::-1]
    # class_rankings[top_scoring_class_index] == 0 etc.
    class_rankings = np.zeros(num_classes, dtype=np.int)
    class_rankings[retrieved_classes] = range(num_classes)
    # Which of these is a true label?
    retrieved_class_true = np.zeros(num_classes, dtype=np.bool)
    retrieved_class_true[class_rankings[pos_class_indices]] = True
    # Num hits for every truncated retrieval list.
    retrieved_cumulative_hits = np.cumsum(retrieved_class_true)
    # Precision of retrieval list truncated at each hit, in order of pos_labels.
    precision_at_hits = (
            retrieved_cumulative_hits[class_rankings[pos_class_indices]] /
            (1 + class_rankings[pos_class_indices].astype(np.float)))
    return pos_class_indices, precision_at_hits


def calculate_per_class_lwlrap(truth, scores):
    """Calculate label-weighted label-ranking average precision.

    Arguments:
      truth: np.array of (num_samples, num_classes) giving boolean ground-truth
        of presence of that class in that sample.
      scores: np.array of (num_samples, num_classes) giving the classifier-under-
        test's real-valued score for each class for each sample.

    Returns:
      per_class_lwlrap: np.array of (num_classes,) giving the lwlrap for each
        class.
      weight_per_class: np.array of (num_classes,) giving the prior of each
        class within the truth labels.  Then the overall unbalanced lwlrap is
        simply np.sum(per_class_lwlrap * weight_per_class)
    """
    assert truth.shape == scores.shape
    num_samples, num_classes = scores.shape
    # Space to store a distinct precision value for each class on each sample.
    # Only the classes that are true for each sample will be filled in.
    precisions_for_samples_by_classes = np.zeros((num_samples, num_classes))
    for sample_num in range(num_samples):
        pos_class_indices, precision_at_hits = (
            _one_sample_positive_class_precisions(scores[sample_num, :],
                                                  truth[sample_num, :]))
        precisions_for_samples_by_classes[sample_num, pos_class_indices] = (
            precision_at_hits)
    labels_per_class = np.sum(truth > 0, axis=0)
    weight_per_class = labels_per_class / float(np.sum(labels_per_class))
    # Form average of each column, i.e. all the precisions assigned to labels in
    # a particular class.
    per_class_lwlrap = (np.sum(precisions_for_samples_by_classes, axis=0) /
                        np.maximum(1, labels_per_class))
    # overall_lwlrap = simple average of all the actual per-class, per-sample precisions
    #                = np.sum(precisions_for_samples_by_classes) / np.sum(precisions_for_samples_by_classes > 0)
    #           also = weighted mean of per-class lwlraps, weighted by class label prior across samples
    #                = np.sum(per_class_lwlrap * weight_per_class)
    return per_class_lwlrap, weight_per_class

In [3]:
def lwlrap(y_pred,y_true):
    score, weight = calculate_per_class_lwlrap(y_true.cpu().numpy(), y_pred.cpu().numpy())
    lwlrap = (score * weight).sum()
    return torch.from_numpy(np.array(lwlrap))

In [4]:
class MixUpCallback(LearnerCallback):
    "Callback that creates the mixed-up input and target."
    def __init__(self, learn:Learner, alpha:float=0.4, stack_x:bool=False, stack_y:bool=True):
        super().__init__(learn)
        self.alpha,self.stack_x,self.stack_y = alpha,stack_x,stack_y
    
    def on_train_begin(self, **kwargs):
        if self.stack_y: self.learn.loss_func = MixUpLoss(self.learn.loss_func)
        
    def on_batch_begin(self, last_input, last_target, train, **kwargs):
        "Applies mixup to `last_input` and `last_target` if `train`."
        if not train: return
        lambd = np.random.beta(self.alpha, self.alpha, last_target.size(0))
        lambd = np.concatenate([lambd[:,None], 1-lambd[:,None]], 1).max(1)
        lambd = last_input.new(lambd)
        shuffle = torch.randperm(last_target.size(0)).to(last_input.device)
        x1, y1 = last_input[shuffle], last_target[shuffle]
        if self.stack_x:
            new_input = [last_input, last_input[shuffle], lambd]
        else: 
            new_input = (last_input * lambd.view(lambd.size(0),1,1,1) + x1 * (1-lambd).view(lambd.size(0),1,1,1))
        if self.stack_y:
            new_target = torch.cat([last_target[:,None].float(), y1[:,None].float(), lambd[:,None].float()], 1)
        else:
            if len(last_target.shape) == 2:
                lambd = lambd.unsqueeze(1).float()
            new_target = last_target.float() * lambd + y1.float() * (1-lambd)
        return {'last_input': new_input, 'last_target': new_target}  
    
    def on_train_end(self, **kwargs):
        if self.stack_y: self.learn.loss_func = self.learn.loss_func.get_old()
        

class MixUpLoss(nn.Module):
    "Adapt the loss function `crit` to go with mixup."
    
    def __init__(self, crit, reduction='mean'):
        super().__init__()
        if hasattr(crit, 'reduction'): 
            self.crit = crit
            self.old_red = crit.reduction
            setattr(self.crit, 'reduction', 'none')
        else: 
            self.crit = partial(crit, reduction='none')
            self.old_crit = crit
        self.reduction = reduction
        
    def forward(self, output, target):
        if len(target.size()) == 2:
            loss1, loss2 = self.crit(output,target[:,0].long()), self.crit(output,target[:,1].long())
            d = (loss1 * target[:,2] + loss2 * (1-target[:,2])).mean()
        else:  d = self.crit(output, target)
        if self.reduction == 'mean': return d.mean()
        elif self.reduction == 'sum':            return d.sum()
        return d
    
    def get_old(self):
        if hasattr(self, 'old_crit'):  return self.old_crit
        elif hasattr(self, 'old_red'): 
            setattr(self.crit, 'reduction', self.old_red)
            return self.crit

def mixup(learn:Learner, alpha:float=0.4, stack_x:bool=False, stack_y:bool=True) -> Learner:
    "Add mixup https://arxiv.org/abs/1710.09412 to `learn`."
    learn.callback_fns.append(partial(MixUpCallback, alpha=alpha, stack_x=stack_x, stack_y=stack_y))
    return learn
Learner.mixup = mixup

In [5]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(out_channels, out_channels, 3, 1, 1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )

        self._init_weights()
        
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.zeros_(m.bias)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = F.avg_pool2d(x, 2)
        return x
    
class Classifier(nn.Module):
    def __init__(self, num_classes=1000): # <======== modificaition to comply fast.ai
        super().__init__()
        
        self.conv = nn.Sequential(
            ConvBlock(in_channels=3, out_channels=64),
            ConvBlock(in_channels=64, out_channels=128),
            ConvBlock(in_channels=128, out_channels=256),
            ConvBlock(in_channels=256, out_channels=512),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1)) # <======== modificaition to comply fast.ai
        self.fc = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(512, 128),
            nn.PReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.1),
            nn.Linear(128, num_classes),
        )

    def forward(self, x):
        x = self.conv(x)
        #x = torch.mean(x, dim=3)   # <======== modificaition to comply fast.ai
        #x, _ = torch.max(x, dim=2) # <======== modificaition to comply fast.ai
        x = self.avgpool(x)         # <======== modificaition to comply fast.ai
        x = self.fc(x)
        return x

In [6]:
def borrowed_model(pretrained=False, **kwargs):
    return Classifier(**kwargs)

In [7]:
DATA = Path('freesound_audio_tagging')
PREPROCESSED = Path('fat2019')
WORK = Path('work')

CSV_TRN_CURATED = DATA/'train_curated.csv'
CSV_TRN_NOISY = DATA/'train_noisy.csv'
CSV_TRN_NOISY_BEST50S = PREPROCESSED/'trn_noisy_best50s.csv'
CSV_SUBMISSION = DATA/'sample_submission.csv'

MELS_TRN_CURATED = PREPROCESSED/'mels_train_curated.pkl'
MELS_TRN_NOISY = PREPROCESSED/'mels_train_noisy.pkl'
MELS_TRN_NOISY_BEST50S = PREPROCESSED/'mels_trn_noisy_best50s.pkl'
MELS_TEST = PREPROCESSED/'mels_test.pkl'

trn_curated_df = pd.read_csv(CSV_TRN_CURATED)
trn_noisy_df = pd.read_csv(CSV_TRN_NOISY)
trn_noisy50s_df = pd.read_csv(CSV_TRN_NOISY_BEST50S)
test_df = pd.read_csv(CSV_SUBMISSION)

In [8]:
X_train_curated = pickle.load(open(MELS_TRN_CURATED, 'rb'))
CUR_X_FILES, CUR_X = list(trn_curated_df.fname.values), X_train_curated

def open_fat2019_image(fn, convert_mode, after_open)->Image:
    # open
    idx = CUR_X_FILES.index(fn.split('/')[-1])
    x = PIL.Image.fromarray(CUR_X[idx])
    # crop 1sec
    time_dim, base_dim = x.size
    crop_x = random.randint(0, time_dim - base_dim)
    x = x.crop([crop_x, 0, crop_x+base_dim, base_dim])    
    # standardize
    return Image(pil2tensor(x, np.float32).div_(255))

vision.data.open_image = open_fat2019_image

In [9]:
SEED = 2000
BS = 128
SIZE = 128

In [10]:
tfms = get_transforms(do_flip=True, max_rotate=0, max_lighting=0.1, max_zoom=0, max_warp=0.)

In [11]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

In [12]:
acc_val = []
c = 1

for train_index, val_index in skf.split(trn_curated_df.index, trn_curated_df['labels']):
        
    src = (ImageList.from_csv(WORK, Path('..')/CSV_TRN_CURATED, folder='trn_curated')
          .split_by_idxs(train_index, val_index)
          .label_from_df(label_delim=',')
          )

    data_fold = (src.transform(tfms, size=SIZE)
            .databunch(bs=BS)
            .normalize(imagenet_stats)
           )
    
    learn = cnn_learner(data_fold, borrowed_model, pretrained=False, metrics=[lwlrap]).mixup(stack_y=False)
    learn.unfreeze()
    
    name = "cv_model_n" + str(c)
    print(name)
    cb = SaveModelCallback(learn, every='improvement', monitor='lwlrap', name=name)
    
    learn.fit_one_cycle(200, 3e-2, callbacks=cb)
    
    c += 1

/opt/conda/envs/fastai/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


cv_model_n1


epoch,train_loss,valid_loss,lwlrap,time
0,0.719232,0.592169,0.095954,00:16
1,0.547657,0.264094,0.095756,00:10
2,0.339267,0.082830,0.115683,00:10
3,0.210375,0.071359,0.173506,00:10
4,0.146445,0.071230,0.174972,00:10
5,0.113210,0.068674,0.208555,00:10
6,0.095137,0.066133,0.247121,00:10
7,0.084608,0.066700,0.228543,00:10
8,0.077890,0.063706,0.266010,00:10
9,0.073726,0.064505,0.237144,00:10


Better model found at epoch 0 with lwlrap value: 0.09595445690546234.
Better model found at epoch 2 with lwlrap value: 0.11568327793046226.
Better model found at epoch 3 with lwlrap value: 0.17350558825007015.
Better model found at epoch 4 with lwlrap value: 0.17497217736781345.
Better model found at epoch 5 with lwlrap value: 0.20855509326520494.
Better model found at epoch 6 with lwlrap value: 0.24712119359364967.
Better model found at epoch 8 with lwlrap value: 0.2660098860481654.
Better model found at epoch 12 with lwlrap value: 0.2855878261620658.
Better model found at epoch 13 with lwlrap value: 0.31411218882233366.
Better model found at epoch 15 with lwlrap value: 0.39578076563711867.
Better model found at epoch 16 with lwlrap value: 0.4208315896519141.
Better model found at epoch 17 with lwlrap value: 0.42413072729280854.
Better model found at epoch 18 with lwlrap value: 0.4464106396736901.
Better model found at epoch 19 with lwlrap value: 0.5002928766651634.
Better model found

epoch,train_loss,valid_loss,lwlrap,time
0,0.702049,0.524839,0.072961,00:12
1,0.521500,0.222207,0.097318,00:11
2,0.314970,0.078279,0.108814,00:11
3,0.195630,0.071728,0.169851,00:11
4,0.137422,0.070444,0.174351,00:11
5,0.107204,0.067301,0.250673,00:11
6,0.090867,0.066106,0.214101,00:11
7,0.081615,0.064491,0.241396,00:11
8,0.075942,0.064367,0.253047,00:11
9,0.072206,0.061460,0.312677,00:11


...

Better model found at epoch 0 with lwlrap value: 0.07579642783210912.
Better model found at epoch 1 with lwlrap value: 0.08592302117114714.
Better model found at epoch 2 with lwlrap value: 0.1189245881681177.
Better model found at epoch 3 with lwlrap value: 0.1580552962665362.
Better model found at epoch 4 with lwlrap value: 0.1761074722907369.
Better model found at epoch 5 with lwlrap value: 0.21232723524994107.
Better model found at epoch 6 with lwlrap value: 0.21846030487388415.
Better model found at epoch 8 with lwlrap value: 0.23791344107499404.
Better model found at epoch 9 with lwlrap value: 0.2529660226129973.
Better model found at epoch 10 with lwlrap value: 0.269987875347379.
Better model found at epoch 11 with lwlrap value: 0.36865540419227605.
Better model found at epoch 12 with lwlrap value: 0.3755584386099416.
Better model found at epoch 14 with lwlrap value: 0.4055014574379853.
Better model found at epoch 17 with lwlrap value: 0.4162950082865245.
Better model found at ep

epoch,train_loss,valid_loss,lwlrap,time
0,0.712179,1.350485,0.065176,00:12
1,0.534141,0.210797,0.100021,00:11
2,0.324649,0.082915,0.125996,00:11
3,0.201281,0.074394,0.153420,00:11
4,0.141002,0.075030,0.164207,00:11
5,0.109867,0.069729,0.190831,00:11
6,0.093194,0.067790,0.208517,00:11
7,0.083708,0.069109,0.243074,00:11
8,0.077981,0.066888,0.207488,00:11
9,0.074511,0.061880,0.301406,00:11


Better model found at epoch 0 with lwlrap value: 0.06517577151579505.
Better model found at epoch 1 with lwlrap value: 0.10002125570136638.
Better model found at epoch 2 with lwlrap value: 0.12599566188276654.
Better model found at epoch 3 with lwlrap value: 0.15342043210462886.
Better model found at epoch 4 with lwlrap value: 0.16420711190576717.
Better model found at epoch 5 with lwlrap value: 0.19083098928364983.
Better model found at epoch 6 with lwlrap value: 0.2085168810355848.
Better model found at epoch 7 with lwlrap value: 0.24307397624210558.
Better model found at epoch 9 with lwlrap value: 0.30140605363298834.
Better model found at epoch 12 with lwlrap value: 0.3342568158255553.
Better model found at epoch 14 with lwlrap value: 0.3967760587683644.
Better model found at epoch 15 with lwlrap value: 0.4131152931771111.
Better model found at epoch 18 with lwlrap value: 0.504053678342411.
Better model found at epoch 19 with lwlrap value: 0.5074070504019133.
Better model found at 

epoch,train_loss,valid_loss,lwlrap,time
0,0.711174,0.658925,0.073569,00:11
1,0.538006,0.190936,0.085710,00:10
2,0.331020,0.090223,0.123571,00:10
3,0.205497,0.078467,0.138380,00:10
4,0.143091,0.078270,0.178642,00:10
5,0.110472,0.075209,0.209991,00:10
6,0.092441,0.077097,0.164342,00:10
7,0.081630,0.068438,0.255321,00:10
8,0.075045,0.073301,0.206410,00:11
9,0.070827,0.068502,0.257271,00:10


Better model found at epoch 0 with lwlrap value: 0.07356856303742917.
Better model found at epoch 1 with lwlrap value: 0.08571031496301315.
Better model found at epoch 2 with lwlrap value: 0.12357083912896422.
Better model found at epoch 3 with lwlrap value: 0.13838003038839225.
Better model found at epoch 4 with lwlrap value: 0.17864231149760262.
Better model found at epoch 5 with lwlrap value: 0.20999086988542648.
Better model found at epoch 7 with lwlrap value: 0.2553206814758026.
Better model found at epoch 9 with lwlrap value: 0.2572709440367417.
Better model found at epoch 10 with lwlrap value: 0.3053631860387906.
Better model found at epoch 12 with lwlrap value: 0.3210335655123564.
Better model found at epoch 13 with lwlrap value: 0.3235480297770296.
Better model found at epoch 15 with lwlrap value: 0.3858498048197163.
Better model found at epoch 17 with lwlrap value: 0.4565976425978235.
Better model found at epoch 18 with lwlrap value: 0.4621287486920661.
Better model found at 

<h2>Stacking generated model</h2>

In [15]:
learn.load("cv_model_n3")
learn.export()